Perform EDA and data cleaning 

In [5]:
import pandas as pd

In [7]:
links =  pd.read_csv("/Users/soledadmusella/Desktop/data project 4/links.csv")
links.head()


,movieId,imdbId,tmdbId
0,1,114709,862.0
1,2,113497,8844.0
2,3,113228,15602.0
3,4,114885,31357.0
4,5,113041,11862.0


In [8]:
movies =  pd.read_csv("/Users/soledadmusella/Desktop/data project 4/movies.csv")
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [11]:
movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9742 entries, 0 to 9741
Data columns (total 3 columns):
movieId    9742 non-null int64
title      9742 non-null object
genres     9742 non-null object
dtypes: int64(1), object(2)
memory usage: 228.4+ KB


In [31]:
ratings =  pd.read_csv("/Users/soledadmusella/Desktop/data project 4/ratings.csv")
ratings.head()


,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [12]:
ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100836 entries, 0 to 100835
Data columns (total 4 columns):
userId       100836 non-null int64
movieId      100836 non-null int64
rating       100836 non-null float64
timestamp    100836 non-null int64
dtypes: float64(1), int64(3)
memory usage: 3.1 MB


In [35]:
df2 = pd.merge(ratings, movies, how='left' , on='movieId')
df2.head()

,userId,movieId,rating,timestamp,title,genres
0,1,1,4.0,964982703,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,1,3,4.0,964981247,Grumpier Old Men (1995),Comedy|Romance
2,1,6,4.0,964982224,Heat (1995),Action|Crime|Thriller
3,1,47,5.0,964983815,Seven (a.k.a. Se7en) (1995),Mystery|Thriller
4,1,50,5.0,964982931,"Usual Suspects, The (1995)",Crime|Mystery|Thriller


In [26]:
tags =  pd.read_csv("/Users/soledadmusella/Desktop/data project 4/tags.csv")
tags.head()


,userId,movieId,tag,timestamp
0,2,60756,funny,1445714994
1,2,60756,Highly quotable,1445714996
2,2,60756,will ferrell,1445714992
3,2,89774,Boxing story,1445715207
4,2,89774,MMA,1445715200


In [13]:
tags.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3683 entries, 0 to 3682
Data columns (total 4 columns):
userId       3683 non-null int64
movieId      3683 non-null int64
tag          3683 non-null object
timestamp    3683 non-null int64
dtypes: int64(3), object(1)
memory usage: 115.2+ KB


In [37]:
df3 = pd.merge(df2, tags, how='left' , on= ["userId",'movieId'],suffixes=("_ratings","_tags"))
df3.head()

,userId,movieId,rating,timestamp_ratings,title,genres,tag,timestamp_tags
0,1,1,4.0,964982703,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,NaN,NaN
1,1,3,4.0,964981247,Grumpier Old Men (1995),Comedy|Romance,NaN,NaN
2,1,6,4.0,964982224,Heat (1995),Action|Crime|Thriller,NaN,NaN
3,1,47,5.0,964983815,Seven (a.k.a. Se7en) (1995),Mystery|Thriller,NaN,NaN
4,1,50,5.0,964982931,"Usual Suspects, The (1995)",Crime|Mystery|Thriller,NaN,NaN


In [44]:
df_nonull = df3[~df3.tag.isnull()].reset_index() #df no nulls 
df_nonull.head()

,index,userId,movieId,rating,timestamp_ratings,title,genres,tag,timestamp_tags
0,241,2,60756,5.0,1445714980,Step Brothers (2008),Comedy,funny,1.445715e+09
1,242,2,60756,5.0,1445714980,Step Brothers (2008),Comedy,Highly quotable,1.445715e+09
2,243,2,60756,5.0,1445714980,Step Brothers (2008),Comedy,will ferrell,1.445715e+09
3,252,2,89774,5.0,1445715189,Warrior (2011),Drama,Boxing story,1.445715e+09
4,253,2,89774,5.0,1445715189,Warrior (2011),Drama,MMA,1.445715e+09


In [59]:
final_df = df3.drop(columns=['timestamp_tags',"timestamp_ratings","genres","title","tag"])
final_df.head()

,userId,movieId,rating
0,1,1,4.0
1,1,3,4.0
2,1,6,4.0
3,1,47,5.0
4,1,50,5.0


In [60]:
from surprise import Reader, Dataset
reader = Reader()
data = Dataset.load_from_df(final_df,reader)

In [62]:
dataset = data.build_full_trainset()
print('Number of users: ', dataset.n_users, '\n')
print('Number of items: ', dataset.n_items)

Number of users:  610 

Number of items:  9724


In [63]:
from surprise.model_selection import cross_validate
from surprise.prediction_algorithms import SVD
from surprise.prediction_algorithms import KNNWithMeans, KNNBasic, KNNBaseline
from surprise.model_selection import GridSearchCV
import numpy as np

In [68]:
## Perform a gridsearch with SVD

params = {'n_factors': [20, 50, 100],
         'reg_all': [0.02, 0.05, 0.1]}
g_s_svd = GridSearchCV(SVD,param_grid=params,n_jobs=-1)
g_s_svd.fit(data)

In [69]:
print(g_s_svd.best_score)
print(g_s_svd.best_params)

{'rmse': 0.8611521086050399, 'mae': 0.6593294240934615}
{'rmse': {'n_factors': 100, 'reg_all': 0.05}, 'mae': {'n_factors': 50, 'reg_all': 0.02}}


In [70]:
knn_basic = KNNBasic(sim_options={'name':'pearson', 'user_based':True})
cv_knn_basic = cross_validate(knn_basic, data, n_jobs=-1)

In [72]:
for i in cv_knn_basic.items():
    print(i)
print(np.mean(cv_knn_basic['test_rmse']))

('test_rmse', array([0.97389352, 0.96929434, 0.96108936, 0.96717172, 0.95585566]))
('test_mae', array([0.75000064, 0.74506663, 0.73463171, 0.74407766, 0.7358883 ]))
('fit_time', (0.6305530071258545, 0.7054548263549805, 0.6517109870910645, 0.7526159286499023, 0.5147759914398193))
('test_time', (1.8803439140319824, 1.8291451930999756, 1.7528021335601807, 1.56485915184021, 1.6515321731567383))
0.9654609214760852
